<a href="https://colab.research.google.com/github/dml2611/Trial/blob/main/ranked_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Packages**

In [ ]:
# Install the following packages
!pip uninstall tweepy
!pip install tweepy
!pip install num2words
!pip install praw           # Python Reddit API Wrapper is a Python module that provides a simple access to Reddit's API
!pip install svgling        # Draw NER Trees / SVG Diagrams / Rendering of linguistics-style constituent trees into SVG

# **Reddit Scrapper**

In [2]:
# Import Libraries
import praw

def reddit_scrapper():
  # Please change with your own client id, client secret, and user agent

  # Initializing the client id, client secret, and user agent
  reddit = praw.Reddit(client_id='NIrpLE8RCFTYCiHBocVL4w', 
                     client_secret='X1ymgznBjeU-H7-noZH4czW9PJ-8qA', 
                     user_agent='Scrapper', 
                     check_for_async = False)

  # Get 5 hot posts from the CryptoCurrency subreddit
  hot_posts = reddit.subreddit('CryptoCurrency').search('bitcoin', sort='top', limit=5)

  # Fetching the reddits
  reddit_posts = []
  for post in hot_posts:
    reddit_posts.append(post.selftext)
  return reddit_posts

# **Twitter Scrapper**

In [3]:
# Importing the libraries
import tweepy
import re
import pickle

from tweepy import OAuthHandler

def twitter_scrapper():
  # Please change with your own consumer key, consumer secret, access token and access secret

  # Initializing the keys
  consumer_key = 'MpEJcCsElugUxpqtUbPKVfDja'
  consumer_secret = 'GBcdBMpB2b0niN23oqrlgafjbpaC2lsWuRHem8Fn9Y6GgjWzB9' 
  access_token = '1353714375605936129-a4B9HUHw33z816KJnmJNfqrxNNHYel'
  access_secret ='LmO1etlQQHPliSOAyzXBOsYkX9D8iScml2JOhhi0KcmHt'

  # Initializing the tokens
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  args = ['bitcoin'];     
  api = tweepy.API(auth,timeout=10)

  # Fetching the tweets
  twitter_posts = []
  query = args[0]
  if len(args) == 1:
    for status in tweepy.Cursor(api.search_tweets, q=query+" -filter:retweets",lang='en',result_type='recent', tweet_mode = 'extended').items(5):
      twitter_posts.append(status.full_text)
  return twitter_posts

# **Data Preprocessing**

In [46]:
#Import Libraries
import re    
from num2words import num2words     #To convert numbers into words in text

#Function to remove emojis
def remove_emoji(string):
  emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)

#Data Cleaning
#stop_words = set(stopwords.words('english'))

def data_preparation(scrapped_posts):
  posts_filtered = []
  for post in scrapped_posts:
    post = re.sub(r"^https://ibb.co/[a-zA-Z0-9]*\s", " ", post)    #Remove links inside reddit posts
    post = re.sub(r"\s+https://ibb.co/[a-zA-Z0-9]*\s", " ", post)      
    post = re.sub(r"(https://ibb.co/[a-zA-Z0-9]*)", " ", post)          
    post = re.sub(r"\s+https://ibb.co/[a-zA-Z0-9]*$", " ", post)        
    post = re.sub(r"^https://t.co/[a-zA-Z0-9]*\s", " ", post)      #Remove links inside twitter posts
    post = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", post)         
    post = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", post)
    post = re.sub("@[A-Za-z0-9_]+","", post)      #Remove mentions
    post = re.sub("#[A-Za-z0-9_]+","", post)         #Remove hashtags
    post = re.compile(r'<[^>]+>').sub(" ", post)        #Remove html tags
    post = re.sub(r"’","'", post)
    post = re.sub(r"that's","that is", post)
    post = re.sub(r"there's","there is", post)
    post = re.sub(r"what's","what is", post)
    post = re.sub(r"where's","where is", post)
    post = re.sub(r"it's","it is", post)
    post = re.sub(r"It's","it is", post)
    post = re.sub(r"I'm","I am", post)
    post = re.sub(r"who's","who is", post)
    post = re.sub(r"i'm","i am", post)
    post = re.sub(r"she's","she is", post)
    post = re.sub(r"he's","he is", post)
    post = re.sub(r"you're","you are", post)
    post = re.sub(r"they're","they are", post)
    post = re.sub(r"who're","who are", post)
    post = re.sub(r"ain't","am not", post)
    post = re.sub(r"wouldn't","would not", post)
    post = re.sub(r"shouldn't","should not", post)
    post = re.sub(r"can't","can not", post)
    post = re.sub(r"couldn't","could not", post)
    post = re.sub(r"won't","will not", post)
    post = re.sub(r"don't","do not", post)
    post = re.sub(r"%"," percent", post)
        #post = re.sub(r"\d"," ", post)        #Remove digits
    post = re.sub(r"\s+[s]\s+"," ", post)
    post = re.sub(r"\s*[\[\]\(\)\-@\*#<>$\'\":]\s*"," ", post)     #Remove special symbols
    post = re.sub(r"\s+"," ", post) #Remove special symbols
    #words = [word for word in nltk.word_tokenize(post) if word.lower() not in stopwords.words('english')]        #Stopword Removal
    #posts_filtered.append(' '.join(words))
    posts_filtered.append(post)
  return posts_filtered

def filter_emojis(posts):
  posts_filtered = []
  for post in posts:
    post = remove_emoji(post)
    posts_filtered.append(post)
  return posts_filtered

def number_to_words(posts):
  num2words_posts = []
  for post in posts:
    numbers = re.findall(r'\d+', post)
    for num in numbers:
      post = re.sub(num, str(num2words(int(num))+" "), post)
    num2words_posts.append(post)
  return num2words_posts

# **POS-Tagging using NLTK**

In [47]:
# Import Libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# POS-Tagger NLTK
def pos_tagger(posts):
  nltk_tagged_sequences = []

  for post in posts:
    dictionary = {}
    word_tags = []
    word_tokens = nltk.word_tokenize(post)
    tagged_words = nltk.pos_tag(word_tokens)
    for tw in tagged_words:
      word_tags.append((tw[0], tw[1]))
    nltk_tagged_sequences.append(word_tags)
  return nltk_tagged_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


# **NER using NLTK** 

In [56]:
# Named Entity Extraction
def entity_extraction_nltk(posts):
  nltk_ner_list = []
  draw_ner_tree_list = []
  for i in range(len(posts)):
    nltk_ner = []
    for chunk in nltk.ne_chunk(posts[i]):
      draw_ner_tree_list.append(nltk.ne_chunk(posts[i]))
      if hasattr(chunk, 'label'):
        nltk_ner.append((chunk.label(), ' '.join(c[0] for c in chunk)))
    nltk_ner_list.append(nltk_ner)
  return nltk_ner_list, draw_ner_tree_list


#Ranked NER Tags
def ranked_frequencies_nltk(nltk_ner_list):
  nltk_frequency = []
  nltk_ranked_frequency = []

  for item in nltk_ner_list:
    dictionary = {'PERSON': 0, 'ORGANIZATION': 0, 'FACILITY': 0, 'GSP': 0, 'LOCATION': 0, 'GPE': 0, 'DATE': 0, 'TIME': 0, 'DURATION' : 0, 'SET': 0, 'MONEY': 0, 'NUMBER': 0, 'ORDINAL': 0, 'PERCENT': 0}
    for ner_tag in item:
      if ner_tag[0] == 'PERSON':
        dictionary['PERSON'] = dictionary['PERSON'] + 1
      elif ner_tag[0] == 'ORGANIZATION':
        dictionary['ORGANIZATION'] = dictionary['ORGANIZATION'] + 1
      elif ner_tag[0] == 'FACILITY':
        dictionary['FACILITY'] = dictionary['FACILITY'] + 1
      elif ner_tag[0] == 'GSP':
        dictionary['GSP'] = dictionary['GSP'] + 1
      elif ner_tag[0] == 'LOCATION':
        dictionary['LOCATION'] = dictionary['LOCATION'] + 1
      elif ner_tag[0] == 'GPE':
        dictionary['GPE'] = dictionary['GPE'] + 1
      elif ner_tag[0] == 'DATE':
        dictionary['DATE'] = dictionary['DATE'] + 1
      elif ner_tag[0] == 'TIME':
        dictionary['TIME'] = dictionary['TIME'] + 1
      elif ner_tag[0] == 'DURATION':
        dictionary['DURATION'] = dictionary['DURATION'] + 1
      elif ner_tag[0] == 'SET':
        dictionary['SET'] = dictionary['SET'] + 1
      elif ner_tag[0] == 'MONEY':
        dictionary['MONEY'] = dictionary['MONEY'] + 1
      elif ner_tag[0] == 'NUMBER':
        dictionary['NUMBER'] = dictionary['NUMBER'] + 1
      elif ner_tag[0] == 'ORDINAL':
        dictionary['ORDINAL'] = dictionary['ORDINAL'] + 1
      elif ner_tag[0] == 'PERCENT':
        dictionary['PERCENT'] = dictionary['PERCENT'] + 1

    nltk_frequency.append(dictionary)

    for dictionary in nltk_frequency:
      nltk_ranked_frequency.append(sorted(dictionary.items(), key=lambda x: x[1], reverse=True))
  return nltk_ranked_frequency

# **NER using Spacy** 

In [57]:
# Import Libraries
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

#Named Entity Extraction using Spacy
def entity_extraction_spacy(posts):
  spacy_ner_list = []
  draw_ner_tree_list = []
  for post in posts:
    tagged_words = nlp(post)
    draw_ner_tree_list.append( nlp(post))
    spacy_ner_list.append([(X.text, X.label_) for X in tagged_words.ents])
  return spacy_ner_list, draw_ner_tree_list

# Ranked NER Tags
def ranked_frequencies_spacy(spacy_ner_list):
  spacy_frequency = []
  spacy_ranked_frequency = []

  for item in spacy_ner_list:
    dictionary = {'PERSON': 0, 'ORGANIZATION': 0, 'FACILITY': 0, 'GSP': 0, 'LOCATION': 0, 'GPE': 0, 'DATE': 0, 'TIME': 0, 'DURATION' : 0, 'SET': 0, 'MONEY': 0, 'NUMBER': 0, 'ORDINAL': 0, 'PERCENT': 0}
    for ner_tag in item:
      if ner_tag[1] == 'PERSON':
        dictionary['PERSON'] = dictionary['PERSON'] + 1
      elif ner_tag[1] == 'ORGANIZATION':
        dictionary['ORGANIZATION'] = dictionary['ORGANIZATION'] + 1
      elif ner_tag[1] == 'FACILITY':
        dictionary['FACILITY'] = dictionary['FACILITY'] + 1
      elif ner_tag[1] == 'GSP':
        dictionary['GSP'] = dictionary['GSP'] + 1
      elif ner_tag[1] == 'LOCATION':
        dictionary['LOCATION'] = dictionary['LOCATION'] + 1
      elif ner_tag[1] == 'GPE':
        dictionary['GPE'] = dictionary['GPE'] + 1
      elif ner_tag[1] == 'DATE':
        dictionary['DATE'] = dictionary['DATE'] + 1
      elif ner_tag[1] == 'TIME':
        dictionary['TIME'] = dictionary['TIME'] + 1
      elif ner_tag[1] == 'DURATION':
        dictionary['DURATION'] = dictionary['DURATION'] + 1
      elif ner_tag[1] == 'SET':
        dictionary['SET'] = dictionary['SET'] + 1
      elif ner_tag[1] == 'MONEY':
        dictionary['MONEY'] = dictionary['MONEY'] + 1
      elif ner_tag[1] == 'NUMBER':
        dictionary['NUMBER'] = dictionary['NUMBER'] + 1
      elif ner_tag[1] == 'ORDINAL':
        dictionary['ORDINAL'] = dictionary['ORDINAL'] + 1
      elif ner_tag[1] == 'PERCENT':
        dictionary['PERCENT'] = dictionary['PERCENT'] + 1

    spacy_frequency.append(dictionary)

    for dictionary in spacy_frequency:
      spacy_ranked_frequency.append(sorted(dictionary.items(), key=lambda x: x[1], reverse=True))
  return spacy_ranked_frequency


In [59]:
# Scrape Twitter
tweets = twitter_scrapper()
print("TWEETS")
print(tweets)
print()

# Preprocess tweets for removing hashtags, mentions, html tags, emojis, etc.
twitter_posts = data_preparation(tweets)
twitter_posts = filter_emojis(twitter_posts)
twitter_posts = number_to_words(twitter_posts)
print("FILTERED TWEETS")
print(twitter_posts)
print()

# POS Tagging
tagged_tweets = pos_tagger(twitter_posts)
print("POS-TAGGED TWEETS")
print(tagged_tweets)
print()

# NER using NLTK
ner_tweets_nltk, draw_ner_tree_nltk = entity_extraction_nltk(tagged_tweets)
print("NER TAGS FOR TWEETS USING NLTK")
print(ner_tweets_nltk)
print()

ranked_ner_tweets_nltk = ranked_frequencies_nltk(ner_tweets_nltk)
print("RANKED NER TAGS FOR TWEETS USING NLTK")
print(ranked_ner_tweets_nltk)
print()

# NER using Spacy
ner_tweets_spacy, draw_ner_tree_spacy = entity_extraction_spacy(twitter_posts)
print("NER TAGS FOR TWEETS USING SPACY")
print(ner_tweets_spacy)
print()

# Scrape Twitter
tweets = twitter_scrapper()
print("TWEETS")
print(tweets)
print()

TWEETS
['Documentary: 48 HOURS in Amsterdam - No CASH, Only BITCOIN https://t.co/nVONYsCEDo via @YouTube', '#Bitcoin Last Price $19433 #BTC 🚀\nDaily Indicators:\n•Variation since 00h00(UTC): +1.19%\n•MACD:\n -MACD Line: -115\n -Signal Line: -145\n\n#Ethereum Last Price $1329 #ETH\n•Variation: +1.18%\n•MACD:\n -MACD Line: -20\n -Signal Line: -29\n\n#Blockchain #Web3\n(2/6)\n303223', 'Beautiful destinations.\nReservations and information in my bio.\nhttps://t.co/3QAVh1UGI6\n#travel #love #home #revolut #bitcoin  #instagood #style  #beautiful #fitness #picoftheday #follow #beautiful #followme  #happy  #girl #lifestyle https://t.co/BeGbqfGmeZ', 'Seen the announcement of that round 1 went live, lets go waagmi! @Przemek0505 @BowTiedBronco @FriscaCal @FoodYoujia @bumskinsx @loop_225 @m_ratsim @LTomosvari @NSEx78DqYM3tVyU @robin55661 @BITCOIN_AVAX @nkbelgin @Vijayanandk2k @Nstorsantos5 https://t.co/PgUvdOZV89', '#Stablecoins/Bitcoin chart. This is nearly a year timeframe going back to the #bit